Tipos de dados, transformações e normalização
=============================================



## DataFrame



Antes de seguir em frente precisamos do `DataFrame` dos dados que iremos tratar. Neste notebook nós mesmos criaremos os dados e armazenaremos eles em um dicionário. Usaremos o conversor de dicionário em `DataFrame` do próprio `pandas`.



In [1]:
import pandas as pd

dicionario_contendo_os_dados = {
    "Nome": ["Platão", "Sócrates", "Descartes", "Nietzsche"],
    "Idade": [80, 71, 53, 55.0],
    "Bigode épico?": [True, True, False, True],
    "Altura": [pd.NA, pd.NA, 1.55, 1.70],
}

df = pd.DataFrame.from_dict(dicionario_contendo_os_dados)

print(df)

        Nome  Idade  Bigode épico? Altura
0     Platão   80.0           True   <NA>
1   Sócrates   71.0           True   <NA>
2  Descartes   53.0          False   1.55
3  Nietzsche   55.0           True    1.7


O objeto `pd.NA` indica para o `pandas` que o dado não está disponível (*not available*). É bastante comum um conjunto de dados ter certas entradas indisponíveis e faz parte do pré-processamento de dados identificar e tratar esse tipo de problema.



## Tipos dos dados



Dados podem ter diversos tipos. Certas informações, como o nome das pessoas, são representados por textos, logo são strings. Números podem ser inteiros, floats, complexos, Booleanos&#x2026;

Vamos ver se o `pandas` conseguiu entender corretamente os tipos dos nossos dados.



In [2]:
print(df.dtypes)

Nome              object
Idade            float64
Bigode épico?       bool
Altura            object
dtype: object


Segundo o `pandas`, a coluna `Nome` tem o tipo de `object`. Este tipo simplesmente está nos dizendo que existem objetos quaisquer de Python dentro dessa coluna (o que não diz muuuuita coisa na verdade&#x2026;). Uma coluna que contenha dados de diferentes tipos sempre será uma coluna do tipo `object`. Uma coluna que só contenha strings acaba sendo uma coluna do tipo `object` por padrão no `pandas`.

A coluna `Idade` é uma coluna com o tipo `float64`, ou seja, ela guarda números reais que são armazenados em 64 bits no seu computador. Ué, mas as idades são todas números inteiros, o que aconteceu aqui? Aconteceu que uma das idades estava escrita como float (você consegue identificar qual?). Quando o `pandas` encontra dados numéricos misturados, ele sempre escolhe o tipo que consegue englobar a todos. Como os inteiros são um subconjunto dos reais, então o `pandas` optou por usar `float` ao invés de `int` para contemplar todos os dados da coluna `Idade`.

A coluna `Bigode épico?` foi corretamente identificada como uma coluna do tipo Booleana.

A coluna `Altura` foi identificada como uma coluna do tipo `object`&#x2026; mas ela só contém números, não é? Na verdade não, ela contém também células vazias representadas pelo `pd.NA`. Para o `pandas` isso significa que existe uma mistura de diferentes tipos de objetos de Python, logo o tipo da coluna é `object`.

DataFrames de `pandas` têm um método que permite eles inferirem &ldquo;automaticamente&rdquo; os tipos de colunas. Vamos ver como ele se comporta no nosso `DataFrame`.



In [3]:
df = df.convert_dtypes()
print(df.dtypes)

Nome              string
Idade              Int64
Bigode épico?    boolean
Altura            object
dtype: object


Há! O método `convert_dtypes` foi sagaz e entendeu que a coluna `Nome` é composta por strings e que a coluna `Idade` contém na verdade apenas números inteiros. Ele converteu a coluna `Bigode Épico?` de `bool` para `boolean`. A diferença é simples: colunas com o tipo `boolean` permitem células vazias usando o `pd.NA`. Colunas do tipo `bool` não permitem. Por fim, a coluna `Altura` permanece com o tipo `object` pois não há nada que possa ser feito, afinal ela de fato é composta por tipos diferentes de dados.

Digamos que você queira alterar o tipo de uma coluna &ldquo;na mão&rdquo;. Isso pode ser feito com o método `astype`. Vamos alterar a coluna `Idade` para o tipo `float`:



In [4]:
df2 = df.copy()
df2["Idade"] = df2["Idade"].astype(float)
print(df2.dtypes)

Nome              string
Idade            float64
Bigode épico?    boolean
Altura            object
dtype: object


Voilà!



## Eliminando dados incompletos



Existem dados incompletos na nossa tabela; não sabemos a altura de Sócrates nem de Platão. A maioria dos algoritmos de Aprendizado de Máquina que iremos estudar não se dão bem com dados incompletos, logo precisamos tratá-los!

A maneira mais simples (mas não necessariamente a melhor!) de se tratar dados incompletos é eliminá-los! DataFrames têm um método muito conveniente para se fazer isso, chamado de `dropna`. Quando usamos esse método devemos pensar se queremos eliminar as *linhas* que têm dados incompletos (eixo 0) ou se queremos eliminar as *colunas* que têm dados incompletos (eixo 1). Os exemplos abaixo são autoexplicativos.



In [5]:
df3 = df.copy()
df3 = df3.dropna(axis=0)
print(df3)

        Nome  Idade  Bigode épico? Altura
2  Descartes     53          False   1.55
3  Nietzsche     55           True    1.7


In [6]:
df4 = df.copy()
df4 = df4.dropna(axis=1)
print(df4)

        Nome  Idade  Bigode épico?
0     Platão     80           True
1   Sócrates     71           True
2  Descartes     53          False
3  Nietzsche     55           True


## Transformações



Parte do tratamento de dados envolve aplicarmos funções nos dados. Chamamos isso de transformação. Quem sabe você tenha uma temperatura em graus Celsius e queira ela em Kelvin. Neste caso, você terá que aplicar uma função que altere o valor da coluna original adicionando 273,15.

Vamos converter a idade dos nossos filósofos de anos para segundos! Note que estamos fazendo cópias do `DataFrame` original para cada alteração. Isso não é obrigatório nem necessário. Fizemos isso aqui para não ter problema na ordem de execução das células deste notebook. Inclusive, se seu conjunto de dados for muito grande, não é uma boa prática ficar criando cópias!



In [7]:
df5 = df.copy()
df5["Idade"] = df5["Idade"] * 365 * 24 * 3600

print(df5)

        Nome       Idade  Bigode épico? Altura
0     Platão  2522880000           True   <NA>
1   Sócrates  2239056000           True   <NA>
2  Descartes  1671408000          False   1.55
3  Nietzsche  1734480000           True    1.7


Agora temos um problema, a coluna `Idade` contém valores muito grandes! Quem sabe seja razoável aplicar um logaritmo aqui!



In [8]:
import numpy as np

df5["Idade"] = np.log10(df5["Idade"])
print(df5)

        Nome     Idade  Bigode épico? Altura
0     Platão  9.401897           True   <NA>
1   Sócrates  9.350065           True   <NA>
2  Descartes  9.223082          False   1.55
3  Nietzsche  9.239169           True    1.7


Faz parte do pré-processamento de dados refletir sobre quais transformações são interessantes/necessárias nos seus dados. Em particular, se você tem alguma grandeza numérica que varia mais que uma ordem de grandeza, considere a possibilidade de trabalhar com o logaritmo dessa grandeza para evitar que os números muito grandes dominem os cálculos em relação aos números muito pequenos.



## Normalização



Um tipo de transformação de dados muito comum é a chamada *normalização*. Muitos algoritmos de aprendizado de máquina <u>requerem</u> que os dados estejam normalizados. Redes neurais artificiais, por exemplo, se beneficiam quando os dados de entrada estão entre $-1$ e $1$. Certos modelos assumem que os dados vêm de uma distribuição com média zero e desvio padrão um e irão se comportar de maneira subótima quando isso não é o caso.

Existem três tipos usuais de normalização para dados numéricos tabulados: normalização padrão (z-score), normalização pelo máximo absoluto e normalização pelo mínimo e máximo.

A <u>normalização padrão</u> altera a média dos dados para zero e o desvio padrão dos dados para um. Ela pode ser calculada com a equação abaixo, onde $\mu$ é a média dos dados $x$, $\sigma$ é o desvio padrão dos dados $x$, $x_i$ é o valor do atributo $x$ do exemplo $i$ e $z_i$ é o valor normalizado do atributo $x$ do exemplo $i$.

$$z_i = \frac{x_i - \mu}{\sigma}$$



In [9]:
media = df['Idade'].mean()
desvio_padrao = df['Idade'].std()

df['Idade_zscore'] = (df['Idade'] - media) / desvio_padrao

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore
0     Platão     80           True   <NA>      1.175689
1   Sócrates     71           True   <NA>       0.48184
2  Descartes     53          False   1.55     -0.905858
3  Nietzsche     55           True    1.7      -0.75167


A <u>normalização pelo máximo absoluto</u> faz com que o valor máximo dos dados seja 1. Para calcular esta normalização basta dividir os dados pelo valor máximo.



In [10]:
maximo = df['Idade'].max()

df['Idade_norm_max_abs'] = df['Idade'] / maximo

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore  Idade_norm_max_abs
0     Platão     80           True   <NA>      1.175689                 1.0
1   Sócrates     71           True   <NA>       0.48184              0.8875
2  Descartes     53          False   1.55     -0.905858              0.6625
3  Nietzsche     55           True    1.7      -0.75167              0.6875


A <u>normalização pelo mínimo e máximo</u> modifica os dados de forma que, após a transformação, os dados tenham valor mínimo de 0 e valor máximo de 1. O valor transformado $m_i$ do exemplo $i$ depende do valor do atributo $x_i$ deste exemplo e do valor mínimo e valor máximo deste atributo considerando todo o dataset $x$:

$$ m_i = \frac{x_i - \min(x_i)}{\max(x) - \min(x)} $$



In [11]:
maximo = df['Idade'].max()
minimo = df['Idade'].min()

df['Idade_norm_min_max'] = (df['Idade'] - minimo) / (maximo - minimo)

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore  Idade_norm_max_abs  \
0     Platão     80           True   <NA>      1.175689                 1.0   
1   Sócrates     71           True   <NA>       0.48184              0.8875   
2  Descartes     53          False   1.55     -0.905858              0.6625   
3  Nietzsche     55           True    1.7      -0.75167              0.6875   

   Idade_norm_min_max  
0                 1.0  
1            0.666667  
2                 0.0  
3            0.074074  


A normalização de dados em experimentos de Aprendizado de Máquina realizados em Python é geralmente feita pelo excelente módulo `scikit-learn`. Para mais informações, veja o [Guia do Usuário](https://scikit-learn.org/stable/modules/preprocessing.html).

